In [83]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)



load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

## Data partition

In [86]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [87]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    370     370 

## Models

In [88]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [89]:
model_rf <- train(x=TrainData, y=TrainClasses, 
               method = "ranger",
               trControl = ctrl) 

+ Fold01.Rep1: min.node.size=16, mtry=45, splitrule=extratrees 
- Fold01.Rep1: min.node.size=16, mtry=45, splitrule=extratrees 
+ Fold01.Rep1: min.node.size=15, mtry=20, splitrule=extratrees 
- Fold01.Rep1: min.node.size=15, mtry=20, splitrule=extratrees 
+ Fold01.Rep1: min.node.size= 1, mtry=35, splitrule=extratrees 
- Fold01.Rep1: min.node.size= 1, mtry=35, splitrule=extratrees 
+ Fold02.Rep1: min.node.size=16, mtry=45, splitrule=extratrees 
- Fold02.Rep1: min.node.size=16, mtry=45, splitrule=extratrees 
+ Fold02.Rep1: min.node.size=15, mtry=20, splitrule=extratrees 
- Fold02.Rep1: min.node.size=15, mtry=20, splitrule=extratrees 
+ Fold02.Rep1: min.node.size= 1, mtry=35, splitrule=extratrees 
- Fold02.Rep1: min.node.size= 1, mtry=35, splitrule=extratrees 
+ Fold03.Rep1: min.node.size=16, mtry=45, splitrule=extratrees 
- Fold03.Rep1: min.node.size=16, mtry=45, splitrule=extratrees 
+ Fold03.Rep1: min.node.size=15, mtry=20, splitrule=extratrees 
- Fold03.Rep1: min.node.size=15, mtry=20

In [90]:
model_rrf <- train(x=TrainData, y=TrainClasses, 
               method = "RRF",
               trControl = ctrl) 

+ Fold01.Rep1: mtry=71, coefReg=0.68066, coefImp=0.23742 
- Fold01.Rep1: mtry=71, coefReg=0.68066, coefImp=0.23742 
+ Fold01.Rep1: mtry=31, coefReg=0.09412, coefImp=0.03743 
- Fold01.Rep1: mtry=31, coefReg=0.09412, coefImp=0.03743 
+ Fold01.Rep1: mtry=75, coefReg=0.70554, coefImp=0.28112 
- Fold01.Rep1: mtry=75, coefReg=0.70554, coefImp=0.28112 
+ Fold02.Rep1: mtry=71, coefReg=0.68066, coefImp=0.23742 
- Fold02.Rep1: mtry=71, coefReg=0.68066, coefImp=0.23742 
+ Fold02.Rep1: mtry=31, coefReg=0.09412, coefImp=0.03743 
- Fold02.Rep1: mtry=31, coefReg=0.09412, coefImp=0.03743 
+ Fold02.Rep1: mtry=75, coefReg=0.70554, coefImp=0.28112 
- Fold02.Rep1: mtry=75, coefReg=0.70554, coefImp=0.28112 
+ Fold03.Rep1: mtry=71, coefReg=0.68066, coefImp=0.23742 
- Fold03.Rep1: mtry=71, coefReg=0.68066, coefImp=0.23742 
+ Fold03.Rep1: mtry=31, coefReg=0.09412, coefImp=0.03743 
- Fold03.Rep1: mtry=31, coefReg=0.09412, coefImp=0.03743 
+ Fold03.Rep1: mtry=75, coefReg=0.70554, coefImp=0.28112 
- Fold03.Rep1:

## Results

In [91]:
print(model_rf)
plot(model_rf)

Random Forest 

740 samples
 77 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 666, 666, 666, 666, 666, 666, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  Accuracy   Kappa    
   1             35    0.9013514  0.8027027
  15             20    0.8959459  0.7918919
  16             45    0.8945946  0.7891892

Tuning parameter 'splitrule' was held constant at a value of extratrees
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 35, splitrule = extratrees
 and min.node.size = 1.


In [92]:
print(model_rrf)
plot(model_rrf) 

Regularized Random Forest 

740 samples
 77 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 666, 666, 666, 666, 666, 666, ... 
Resampling results across tuning parameters:

  mtry  coefReg     coefImp     Accuracy   Kappa    
  31    0.09411981  0.03742977  0.8135135  0.6270270
  71    0.68065712  0.23742081  0.8702703  0.7405405
  75    0.70553654  0.28112463  0.8729730  0.7459459

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 75, coefReg = 0.7055365
 and coefImp = 0.2811246.


In [93]:
# collect resamples
results <- resamples(list(ranger=model_rf,rrf=model_rrf))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: ranger, rrf 
Number of resamples: 10 

Accuracy 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
ranger 0.8783784 0.8783784 0.8918919 0.9013514 0.9256757 0.9459459    0
rrf    0.7972973 0.8277027 0.8783784 0.8729730 0.9054054 0.9459459    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
ranger 0.7567568 0.7567568 0.7837838 0.8027027 0.8513514 0.8918919    0
rrf    0.5945946 0.6554054 0.7567568 0.7459459 0.8108108 0.8918919    0


## Prediction

In [94]:
pred_rf <- predict(model_rf,TestData)  
pred_rf.prob <- predict(model_rf,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_rf) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      29      13
   NotFast   53     223
                                          
               Accuracy : 0.7925          
                 95% CI : (0.7437, 0.8357)
    No Information Rate : 0.7421          
    P-Value [Acc > NIR] : 0.02163         
                                          
                  Kappa : 0.3551          
                                          
 Mcnemar's Test P-Value : 1.582e-06       
                                          
            Sensitivity : 0.35366         
            Specificity : 0.94492         
         Pos Pred Value : 0.69048         
         Neg Pred Value : 0.80797         
             Prevalence : 0.25786         
         Detection Rate : 0.09119         
   Detection Prevalence : 0.13208         
      Balanced Accuracy : 0.64929         
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
          0.35365854           0.94491525           0.69047619 
      Neg Pred Value            Precision               Recall 
          0.80797101           0.69047619           0.35365854 
                  F1           Prevalence       Detection Rate 
          0.46774194           0.25786164           0.09119497 
Detection Prevalence    Balanced Accuracy 
          0.13207547           0.64928690

In [95]:
roc_obj <- roc(TestClasses, as.numeric(pred_rf))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [96]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_rf))

Data: as.numeric(pred_rf) in 42 controls (TestClasses fast) < 276 cases (TestClasses NotFast).
Area under the curve: 0.7492

In [97]:
pred_rf.prob

fast,NotFast
0.040,0.960
0.116,0.884
0.168,0.832
0.262,0.738
0.210,0.790
0.692,0.308
0.312,0.688
0.484,0.516
0.856,0.144
0.540,0.460
